<a href="https://colab.research.google.com/github/Baheback/RepositorioDelMal/blob/main/proyectofinal_crypto_analisys_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas numpy matplotlib ta
from google.colab import drive
drive.mount('/content/drive')

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Lista de criptomonedas
criptos = {
    "BTC-USD": "Bitcoin",
    "ETH-USD": "Ethereum",
    "BNB-USD": "BNB",
    "XRP-USD": "Ripple",
    "ADA-USD": "Cardano",
    "DOGE-USD": "Dogecoin",
    "SOL-USD": "Solana",
    "DOT-USD": "Polkadot",
    "MATIC-USD": "Polygon",
    "LTC-USD": "Litecoin"
}

# Umbrales de RSI
rsi_buy = 30
rsi_sell = 70

def analizar_gradio(ticker):
    try:
        nombre = criptos.get(ticker, ticker)

        # Descargar datos
        data = yf.download(ticker, period="30d", interval="1d", auto_adjust=True, progress=False)
        if data.empty:
            return "❌ No se pudieron obtener datos de la criptomoneda.", None

        # Asegurar que las columnas sean simples (no MultiIndex)
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        # Verificar que 'Close' exista
        if "Close" not in data.columns:
            return "❌ Error: los datos descargados no contienen la columna 'Close'.", None

        # Asegurarse de que sea una Serie 1D
        close_prices = data["Close"].astype(float).squeeze()

        # Verificar cantidad de datos para análisis técnico
        if len(close_prices) < 15:
            return "⚠️ No hay suficientes datos para análisis técnico.", None

        # Cálculo de indicadores
        rsi = RSIIndicator(close=close_prices, window=14).rsi()
        macd = MACD(close=close_prices)
        data["RSI"] = rsi
        data["MACD"] = macd.macd()
        data["Signal"] = macd.macd_signal()
        data["Tendencia"] = np.where(data["MACD"] > data["Signal"], "Alcista", "Bajista")

        # Último valor
        ultimo = data.iloc[-1]
        precio_actual = float(ultimo["Close"])
        rsi_actual = float(ultimo["RSI"])
        tendencia_actual = ultimo["Tendencia"]

        # Recomendación
        if rsi_actual < rsi_buy and tendencia_actual == "Alcista":
            decision = "Comprar"
        elif rsi_actual > rsi_sell and tendencia_actual == "Bajista":
            decision = "Vender"
        else:
            decision = "Mantener"

        # Crear gráfico
        plt.figure(figsize=(10, 4))
        plt.plot(data.index, data["Close"], label="Precio de cierre", color="blue")
        plt.title(f"Histórico de precios - {nombre}")
        plt.xlabel("Fecha")
        plt.ylabel("Precio (USD)")
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        grafico_path = "grafico.png"
        plt.savefig(grafico_path)
        plt.close()

        # Resultado
        resultado = (
            f"📈 Criptomoneda: {nombre} ({ticker})\n"
            f"📅 Fecha: {ultimo.name.strftime('%Y-%m-%d')}\n"
            f"💰 Precio actual: {precio_actual:.2f} USD\n"
            f"📉 RSI: {rsi_actual:.2f}\n"
            f"📊 Tendencia: {tendencia_actual}\n"
            f"📌 Recomendación: {decision}\n\n"
            "⏳ Revisando RSI y MACD... Esperando 5 minutos hasta el próximo análisis...\n"
        )

        return resultado, grafico_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None

# Gradio
opciones = list(criptos.keys())

interfaz = gr.Interface(
    fn=analizar_gradio,
    inputs=gr.Dropdown(choices=opciones, label="Selecciona una criptomoneda"),
    outputs=[
        gr.Textbox(label="Resultado del análisis"),
        gr.Image(type="filepath", label="Gráfico de precios")
    ],
    title="📊 Análisis Técnico de Criptomonedas",
    description="Usa RSI y MACD para obtener una recomendación de compra, venta o mantener.",
)

interfaz.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e3b5bb2c4602b13d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
